In [1]:
import json

In [6]:
# import torch

In [7]:
with open("/data02/MAG_PROCESSED_DATA/2019FILE/TITLE_SEP_ABSTRACT.txt.3", "r") as fp:
    paper_texts = json.load(fp)

In [8]:
len(paper_texts)

2155965

In [9]:
from sentence_transformers import SentenceTransformer


In [10]:
model = SentenceTransformer('allenai-specter')

In [11]:
2155965/300000

7.18655

In [12]:
import time

In [13]:
import numpy as np

In [14]:
from tqdm import tqdm

In [15]:
import GPUtil

In [16]:
# start = time.time()
# corpus_embeddings = model.encode(paper_texts[:10000], convert_to_tensor=True)
# embeddings = corpus_embeddings.cpu().numpy()
# np.save('/data02/MAG_PROCESSED_DATA/SPECTER_VECTOR/sBERT_embedding.npy.0',embeddings)
# del embeddings
# del corpus_embeddings
# print(time.time()-start)

In [23]:
GPUtil.showUtilization()

| ID | GPU  | MEM |
-------------------
|  0 | 100% | 31% |
|  1 |   0% |  9% |
|  2 | 100% | 26% |
|  3 |   0% | 31% |


In [22]:
def get_vectors(paper_texts_temp):
    corpus_embeddings = model.encode(paper_texts_temp, convert_to_tensor=True)
    embeddings = corpus_embeddings.cpu().numpy()
    
    del corpus_embeddings
    torch.cuda.empty_cache()
    return embeddings

# GET lowest gpu memory

In [1]:
import subprocess, re, os, sys

# GPU picking
# http://stackoverflow.com/a/41638727/419116
# Nvidia-smi GPU memory parsing.
# Tested on nvidia-smi 370.23

def run_command(cmd):
    """Run command, return output as string."""
    
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]
    return output.decode("ascii")

def list_available_gpus():
    """Returns list of available GPU ids."""
    
    output = run_command("nvidia-smi -L")
    # lines of the form GPU 0: TITAN X
    gpu_regex = re.compile(r"GPU (?P<gpu_id>\d+):")
    result = []
    for line in output.strip().split("\n"):
        m = gpu_regex.match(line)
        assert m, "Couldnt parse "+line
        result.append(int(m.group("gpu_id")))
    return result

def gpu_memory_map():
    """Returns map of GPU id to memory allocated on that GPU."""

    output = run_command("nvidia-smi")
    gpu_output = output[output.find("GPU Memory"):]
    # lines of the form
    # |    0      8734    C   python                                       11705MiB |
    memory_regex = re.compile(r"[|]\s+?(?P<gpu_id>\d+)\D+?(?P<pid>\d+).+[ ](?P<gpu_memory>\d+)MiB")
    rows = gpu_output.split("\n")
    result = {gpu_id: 0 for gpu_id in list_available_gpus()}
    for row in gpu_output.split("\n"):
        m = memory_regex.search(row)
        if not m:
            continue
        gpu_id = int(m.group("gpu_id"))
        gpu_memory = int(m.group("gpu_memory"))
        result[gpu_id] += gpu_memory
    return result

def pick_gpu_lowest_memory():
    """Returns GPU with the least allocated memory"""

    memory_gpu_map = [(memory, gpu_id) for (gpu_id, memory) in gpu_memory_map().items()]
    best_memory, best_gpu = sorted(memory_gpu_map)[0]
    return best_gpu

def setup_one_gpu():
    assert not 'tensorflow' in sys.modules, "GPU setup must happen before importing TensorFlow"
    gpu_id = pick_gpu_lowest_memory()
    print("Picking GPU "+str(gpu_id))
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

def setup_no_gpu():
    if 'tensorflow' in sys.modules:
        print("Warning, GPU setup must happen before importing TensorFlow")
    os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [3]:
pick_gpu_lowest_memory()

1

In [2]:
gpu_memory_map()

{0: 3355, 1: 1013, 2: 2813, 3: 3455}

In [4]:
setup_one_gpu()

Picking GPU 1


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [19]:
import torch

In [ ]:
for n in tqdm(range(8)):
    

    print(n)
    embeddings = get_vectors(paper_texts[n*300000: (n+1)*300000])
                  
    directory = '/data02/MAG_PROCESSED_DATA/2019FILE/sBERT_vector_2019_' + str(n+20)
    
    np.save(directory,embeddings)
    
              
    del embeddings
    del directory
    torch.cuda.empty_cache()
    
    GPUtil.showUtilization()
    


  0%|          | 0/8 [00:00<?, ?it/s]

0


 12%|█▎        | 1/8 [49:09<5:44:04, 2949.28s/it]

| ID | GPU  | MEM |
-------------------
|  0 | 100% | 30% |
|  1 |   0% | 24% |
|  2 | 100% | 26% |
|  3 |   0% | 31% |
1


 25%|██▌       | 2/8 [1:38:27<4:55:11, 2951.99s/it]

| ID | GPU  | MEM |
-------------------
|  0 | 100% | 30% |
|  1 |   7% | 24% |
|  2 | 100% | 26% |
|  3 |   0% | 31% |
2


 38%|███▊      | 3/8 [2:27:48<4:06:13, 2954.61s/it]

| ID | GPU  | MEM |
-------------------
|  0 | 100% | 30% |
|  1 |   0% | 24% |
|  2 |  99% | 34% |
|  3 |   0% | 31% |
3


In [ ]:
# PID 155694

In [20]:
del corpus_embeddings

NameError: name 'corpus_embeddings' is not defined